In [33]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib

pd.set_option('display.max_columns', None)

In [34]:
#### FILES

dg = "https://feeds.datagolf.com/preds/fantasy-projection-defaults?tour=pga&site=draftkings&slate=main&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
fantrax = r"C:\Users\mikej\Downloads\Fantrax-Players-Fantasy TOUR 106 (29).csv"

In [35]:
team_color={
     "Philly919": 'rgb(14,195,210)',
     "unit_circle": 'rgb(194,139,221)',
     "AlphaWired": 'rgb(247,160,93)',
     "Sneads Foot": 'rgb(70,214,113)',
     "New Team 4": 'rgb(247,94,56)',
     "Team Gamble": 'rgb(38,147,190)',
     "txmoonshine": 'rgb(219,197,48)',
     "Putt Pirates": 'rgb(115,112,106)'}

active_color={
    "Active":'rgb(146,146,143)',
    "Reserve":'rgb(220,222,202)'
    }

teams_dict = {'919':'Philly919','u_c':'unit_circle',
              'NT 4':'New Team 4','NT 8':'Sneads Foot',
              'txms':'txmoonshine','MG':'Team Gamble',
              'grrr':'Putt Pirates','[AW]':'AlphaWired'}

In [42]:
#fx rosters

teams = pd.read_csv(fantrax)# ,usecols=['Player','Status','Roster Status'])
# teams.columns = ['player','team','active_reserve']
teams['Status'] = teams['Status'].astype('str')
teams_dict = {'919':'Philly919','u_c':'unit_circle','NT 8':'Sneads Foot','NT 4':'New Team 4','txms':'txmoonshine','MG':'Team Gamble','grrr':'Putt Pirates','[AW]':'AlphaWired','FA':'Free Agent'}
teams['Team'] = teams.Status.map(teams_dict)
teams.set_index('Player',inplace=True)

# datagolf dk projections
dg_proj = pd.read_csv(dg,usecols=['player_name','proj_points_total'])
dg_proj = dg_proj[['player_name','proj_points_total']]
dg_proj.columns = ['player','proj_pts']

def fix_long_names(dg_proj):
        #flip first and last name
        names = dg_proj['player'].str.split(expand=True)                  
        names[0] = names[0].str.rstrip(",")
        names[1] = names[1].str.rstrip(",")
        names['player'] = names[1] + " " + names[0]
        # specific player to fix
        names['player'] = np.where(names['player']=='Matt Fitzpatrick', 'Matthew Fitzpatrick', names['player'])
        names['player'] = np.where(names['player']=='Si Kim', 'Si Woo Kim', names['player'])
        names['player'] = np.where(names['player']=='Min Lee', 'Min Woo Lee', names['player'])
        names['player'] = np.where(names['player']=='Byeong An', 'Byeong Hun An', names['player'])
        names['player'] = np.where(names['player']=='Rooyen Van', 'Erik Van Rooyen', names['player'])
        return names.player

# set index to new names
dg_proj.set_index(fix_long_names(dg_proj),inplace=True)

# merge fantrax rosters with datagolf projections
week = pd.merge(teams,dg_proj, left_index=True, right_index=True)#.reset_index()
week[['player','Team','Status']] = week[['player','Team','Status']].astype('string')
week.sort_values('proj_pts',ascending=False,inplace=True)
week

,ID,Team,Position,RkOv,Status,FPts,FP/G,Ros,+/-,Par,Bir,Eag,Bog,DBo,Alb,H1,TP,Oth,Plc,player,proj_pts
Xander Schauffele,*0401u*,AlphaWired,NaN,49,[AW],538.5,89.75,100%,0%,261,109,4,36,3,0,0,6,1,97,"Schauffele, Xander",74.03
Sam Burns,*03nur*,AlphaWired,NaN,78,[AW],526.0,87.67,100%,0%,248,114,5,38,7,0,0,6,2,104,"Burns, Sam",62.49
Justin Thomas,*01rp9*,New Team 4,NaN,111,NT 4,416.5,69.42,100%,0%,199,97,1,42,3,0,0,6,0,159,"Thomas, Justin",60.14
Sungjae Im,*0452x*,Philly919,NaN,75,919,452.5,56.56,100%,0%,316,101,2,61,6,0,0,8,0,398,"Im, Sungjae",58.44
Jordan Spieth,*01uh8*,AlphaWired,NaN,134,[AW],276.5,69.12,100%,0%,160,65,1,41,3,0,0,4,0,149,"Spieth, Jordan",58.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ben Taylor,*03rlp*,Free Agent,NaN,110,FA,228.0,32.57,5%,-0.5%,193,55,0,49,8,0,0,7,1,496,"Taylor, Ben",24.12
Kevin Kisner,*01q9b*,Free Agent,NaN,186,FA,86.0,28.67,3%,-0.5%,83,20,0,15,7,0,0,3,1,219,"Kisner, Kevin",22.40
Ryan Brehm,*01ozp*,Free Agent,NaN,137,FA,203.0,33.83,1%,-0.5%,162,45,2,54,5,0,0,6,2,378,"Brehm, Ryan",21.95
Greg Koch,*05t53*,Free Agent,NaN,279,FA,0.0,0.00,0%,0%,0,0,0,0,0,0,0,0,0,0,"Koch, Greg",21.94


In [43]:
px.bar(week[week.Status=='FA'].sort_values('proj_pts',ascending=False)[:20],
      y='proj_pts',
      log_y=True,
      text_auto='.2s',
      color='proj_pts',
      template='plotly_white')

In [45]:
####   CURRENT WEEK INPUTS   ####
current_week = 11
tournament = "Valspar<br>Championship"
m1 = ['Putt Pirates','unit_circle']
m2 = ['txmoonshine','AlphaWired']
m3 = ['Team Gamble','Philly919']
m4 = ['New Team 4','Sneads Foot']   

In [46]:
def get_matchup_bar(week,matchup):
        matchup_bar = px.bar(week[(week.active_reserve=='Active') & (week.team.isin(matchup))].drop(columns='player').sort_values(by = 'proj_pts',ascending=False).reset_index(),
                             y = 'proj_pts',
                             color = 'team',
                             color_discrete_map=team_color,
                             labels = {'index':"", 'proj_pts':''},
                             text='player',
                             template = 'plotly_dark',
                             hover_name='proj_pts',
                             height=250,
                             log_y=True
                             ).update_xaxes(showticklabels=False
                             ).update_yaxes(tickvals=[50,60,70,80,90,100], tickfont=dict(color='#5A5856')
                             ).update_yaxes(gridcolor="#B1A999", tickfont=dict(color='#5A5856')
                             ).update_layout(legend=dict(orientation='h',title='',y=1.25,x=.2,font_color='#5A5856'))
        return matchup_bar

In [55]:
px.bar(week.reset_index(),
       x='Team',
       y='FPts',
       color='Status',
       hover_name='player')

In [21]:
get_matchup_bar(week,m2)

AttributeError: 'DataFrame' object has no attribute 'active_reserve'

In [26]:
#  TOP 20 PLAYS
top20 = week.sort_values(by = 'proj_pts',ascending=False)[:20].reset_index(drop=True)
px.bar(top20,
        y = 'proj_pts',
        # color = 'team',
        color_discrete_map=team_color,
        labels = {'index':"", 'proj_pts':'Projected Pts'},
        text=week[:20].index,
        #       text_auto = ",.0f",
        template = 'plotly_white',
        height=400,
        log_y=True,
        hover_name='player'
        ).add_hline(y=week.proj_pts.mean(),line_color='darkslategrey'
        ).update_xaxes(showticklabels=False
        ).update_yaxes(showgrid=False)

In [28]:
# ALL PLAYER PROJECTIONS FOR THE WEEK WITH ACTIVE/INACTIVE CALLOUT

fig = px.bar(week.sort_values(by = 'proj_pts',ascending=False).drop(columns='player').reset_index(),#[:47],
       y = 'proj_pts',
       template='plotly_white',
       title='Arnold Palmer Invitational Players on Rosters',
#        hover_name = 'player',
       color = 'Status',
       color_discrete_map=active_color,height=450,
       labels = {'_index':"", 'proj_pts':'Projected Pts'}).add_hline(y=week.proj_pts.mean()).update_xaxes(showticklabels=False).update_layout(title_x=.5)
fig.show()


In [30]:
week.reset_index(drop=True)

,ID,Team,Position,RkOv,Status,FPts,FP/G,Ros,+/-,Par,Bir,Eag,Bog,DBo,Alb,H1,TP,Oth,Plc,player,proj_pts
0,*0401u*,<NA>,NaN,49,[AW],538.5,89.75,100%,0%,261,109,4,36,3,0,0,6,1,97,"Schauffele, Xander",74.03
1,*03nur*,<NA>,NaN,78,[AW],526.0,87.67,100%,0%,248,114,5,38,7,0,0,6,2,104,"Burns, Sam",62.49
2,*01rp9*,<NA>,NaN,111,NT 4,416.5,69.42,100%,0%,199,97,1,42,3,0,0,6,0,159,"Thomas, Justin",60.14
3,*0452x*,<NA>,NaN,75,919,452.5,56.56,100%,0%,316,101,2,61,6,0,0,8,0,398,"Im, Sungjae",58.44
4,*01uh8*,<NA>,NaN,134,[AW],276.5,69.12,100%,0%,160,65,1,41,3,0,0,4,0,149,"Spieth, Jordan",58.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,*03rlp*,<NA>,NaN,110,FA,228.0,32.57,5%,-0.5%,193,55,0,49,8,0,0,7,1,496,"Taylor, Ben",24.12
141,*01q9b*,<NA>,NaN,186,FA,86.0,28.67,3%,-0.5%,83,20,0,15,7,0,0,3,1,219,"Kisner, Kevin",22.40
142,*01ozp*,<NA>,NaN,137,FA,203.0,33.83,1%,-0.5%,162,45,2,54,5,0,0,6,2,378,"Brehm, Ryan",21.95
143,*05t53*,<NA>,NaN,279,FA,0.0,0.00,0%,0%,0,0,0,0,0,0,0,0,0,0,"Koch, Greg",21.94


In [29]:
# histogram showing active/inactive

cuts_made_hist = px.histogram(week.reset_index(drop=True),
                              x='proj_pts',
                              opacity=.7,
                              nbins=8, 
                              color = 'Status',
                              color_discrete_map=active_color,
                              barmode='stack',
                              text_auto=True,
                              height=400,
                              title='Projected Points Distribution<br>by active/reserve',
                              template='plotly_white',
                              width=600,
                              labels={'proj_pts':'Projected Points'}
                              ).update_yaxes(showgrid=False
                              ).update_xaxes(tickmode='array',tickvals=[25,35,45,55,65,75],ticktext=["30-39","40-49", "50-59", "60-69", "70-79", "80-89"]
                              ).update_layout(title_x=.23,legend=dict(title="",x=.75))
cuts_made_hist.show()

In [49]:
# MAKE DATAFRAME WITH ONLY TOP 6 PROJECTED PLAYERS BY TEAM

top_6_proj = pd.DataFrame()

for team in week.team.unique():
    temp = week[week.team==team][['team','player','proj_pts','active_reserve']].sort_values(by='proj_pts',ascending=False)[:6]
    top_6_proj = pd.concat([top_6_proj,temp])
    top_6_proj = top_6_proj.sort_values('proj_pts',ascending=False).reset_index(drop=True)

print("Of the 48 Top Plays...")
print(top_6_proj.active_reserve.value_counts())
top_6_proj.sort_values('proj_pts',ascending=False).reset_index(drop=True).head()

Of the 48 Top Plays...
Active     29
Reserve    16
Name: active_reserve, dtype: Int64


,team,player,proj_pts,active_reserve
0,AlphaWired,"Schauffele, Xander",74.03,Active
1,AlphaWired,"Burns, Sam",62.49,Active
2,New Team 4,"Thomas, Justin",60.14,Active
3,Philly919,"Im, Sungjae",58.44,Active
4,AlphaWired,"Spieth, Jordan",58.29,Active


In [50]:
# PLAYER OPTIONS FOR EACH TEAM THIS WEEK

for team in week.team.unique():
    fig = px.bar(week[week.team == team].sort_values(by='proj_pts',ascending=False), 
                 x = 'player', 
                 y = 'proj_pts', 
                 height=350,
                 width=600,
                 title = f"Choices for {team}", 
                 color='active_reserve', 
                 text_auto='.2s',
                 template = 'plotly_dark',
                 color_discrete_map=active_color,
                 labels = {'proj_pts':'Projected Points','player':"Eligible Players"})
    fig.show()